# Import Dataset

In [1]:
import pandas as pd
import re
import requests

# https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020/data
dataset = pd.read_csv(r"marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv")
dataset.head(5)

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


# Create and Save Priority Products to Collect GTINs For

In [53]:
# Function to extract brand from URL
def extract_brand(url):
    match = re.search(r'amazon\.com/([A-Z]+)(?![a-z])', url)
    if match:
        return match.group(1)
    else:
        return None


# Apply function to create new column
dataset['Brand'] = dataset['Product Url'].apply(extract_brand)
dataset[['Product Url','Brand']]

priority = dataset[
    (dataset['Brand'].isna()==False)&
    (dataset['Model Number'].isna()==False)&
    (dataset['Upc Ean Code'].isna()==True)
    ][['Brand','Model Number','Product Name']]

priority.reset_index(inplace=True, drop=True)
priority.columns = ["brand","mpn","title"]
priority = priority.sort_values('mpn', ascending=False).drop_duplicates(subset='mpn')
priority.to_csv('priority.csv', index=False)
priority.head()

,brand,mpn,title
355,DIAMOND,marvel e dc icons,DIAMOND SELECT TOYS Marvel Milestones: Avenger...
251,Z,ZM7101,Pandemic
130,Z,ZM4900,Onirim
656,ZVEZDA,Z5044,Zvezda 5044 - Soviet Tank Destroyer SU-100 - P...
391,SQUIRREL,Z-1014-Squirrel Lovers,SQUIRREL LOVERS Parking Sign hunter rodent par...


# Load Priority Products to Collect GTINs For

In [22]:
priority = pd.read_csv('priority.csv', index_col=False)
priority.head()

,brand,mpn,title
0,VATERRA,VTR03090,Vaterra 1/10 1972 Chevrolet K10 Pickup Ascende...
1,WWE,US610203,WWE Massive Belt Banners - Airnormous WWE Unit...
2,SONGMICS,UGKR48WT,SONGMICS UGKR48WT Children’s Toy Storage Unit ...
3,WWE,TMLWWE2,WWE TeenyMates Series 2 Locker Room Set
4,THD,TH-6133,THD TH-6133 White Large Indoor or Outdoor Ther...


# Load Logs Already Requested

In [23]:
log = pd.read_csv("logs.csv", index_col=False)
log.head()

,brand,mpn,title,api_response,gtin,gtin_exists


# Create List of Products not yet run through Produkter

In [24]:
merged_df = pd.merge(priority, log, left_on='mpn', right_on='mpn', how='left', indicator=True)

# Filter rows that exist in the first dataframe but not in the second
result_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the '_merge' column as it's no longer needed
result_df = result_df.drop(columns=['_merge'])

# Drop duplicate '_y' columns
result_df = result_df.loc[:, ~result_df.columns.str.endswith('_y')]

# Rename remaining columns to remove '_x' suffixes
todo = result_df.rename(columns=lambda x: x[:-2] if x.endswith('_x') else x)
todo.head()

,brand,mpn,title,api_response,gtin,gtin_exists
0,VATERRA,VTR03090,Vaterra 1/10 1972 Chevrolet K10 Pickup Ascende...,NaN,NaN,NaN
1,WWE,US610203,WWE Massive Belt Banners - Airnormous WWE Unit...,NaN,NaN,NaN
2,SONGMICS,UGKR48WT,SONGMICS UGKR48WT Children’s Toy Storage Unit ...,NaN,NaN,NaN
3,WWE,TMLWWE2,WWE TeenyMates Series 2 Locker Room Set,NaN,NaN,NaN
4,THD,TH-6133,THD TH-6133 White Large Indoor or Outdoor Ther...,NaN,NaN,NaN


# Format Functions

In [18]:
def get_gtin_values(response_json, q, gtin_key='gtin'):
    try:
        gtin_values = response_json.get(q, {}).get(gtin_key, [])
        
        # Filter out None values and strings with length < 5 characters
        filtered_values = [value for value in gtin_values if value is not None and (not isinstance(value, str) or len(value) > 5)]
    
    except: # no response_json
        filtered_values = []    

    return filtered_values
# get_gtin_values(data, q, gtin_key='gtin12')


def filter_strings(*lists):
    result = []
    for lst in lists:
        for item in lst:
            if isinstance(item, str) and len(item) > 5:
                result.append(item)
    return result


def any_list_non_empty(*lists):
    for l in lists:
        if l:
            return True
    return False

# Collect GTINs

In [6]:
url = "https://produkter.p.rapidapi.com/schemas"

headers = {
	"X-RapidAPI-Key": "XXXXXXXXX",
	"X-RapidAPI-Host": "produkter.p.rapidapi.com"
}

In [25]:
# Initialize `log` column variables for each `todo` row
i_brand = None
i_mpn = None
i_title = None
i_api_response = None
i_gtin = None
i_gtin8 = None
i_gtin12 = None
i_gtin13 = None
i_gtins = None
i_gtin_exists = None

# Initialize a list to store rows
output_rows = []

for i, row in todo.iloc[:10, :].iterrows():
    
    i_brand = row['brand']
    i_mpn = row['mpn']
    i_title = row['title']
    print(i_brand,i_mpn)

    querystring = {"q": f"{i_brand} {i_mpn}"}

    # try requests + store i_api_response
    try:
        # Set a timeout of 25 seconds for the request
        response = requests.get(url, headers=headers, params=querystring, timeout=35)
        try:
            response_json = response.json()
            i_api_response = response.status_code
        except Exception as e:
            response_json = None
            i_api_response = e
            
    except requests.Timeout:
        # Request timed out
        response_json = None
        i_api_response = "timeout"

        

    # get gtins
    try:
        i_gtin = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin')
        i_gtin8 = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin8')
        i_gtin12 = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin12')
        i_gtin13 = get_gtin_values(response_json, q=f"{i_brand} {i_mpn}", gtin_key='gtin13')
        
        print(i_gtin,i_gtin8,i_gtin12,i_gtin13)
        
        i_gtins = filter_strings(i_gtin,i_gtin8,i_gtin12,i_gtin13)

        i_gtin_exists = any_list_non_empty(i_gtins)

    except Exception as e:
        # Handle any errors here
        print(f"Error processing gtins on row {i} ({i_brand} {i_mpn}): \n {e}")
        i_gtin = []
        i_gtin8 = []
        i_gtin12 = []
        i_gtin13 = []
        

        i_gtins = filter_strings(i_gtin,i_gtin8,i_gtin12,i_gtin13)

        i_gtin_exists = any_list_non_empty(i_gtins)

    
    # make new rows
    new_row = {
        'brand':i_brand,
        'mpn':i_mpn,
        'title':i_title,
        'api_response':i_api_response,
        'gtins':i_gtins,
        'gtin_exists':i_gtin_exists
    }

    # Append the dictionary to the list
    output_rows.append(new_row)

VATERRA VTR03090
[] [] [] []
WWE US610203
[] [] [] []
SONGMICS UGKR48WT
[] [] [] []
WWE TMLWWE2
[] [] [] ['733947120073']
THD TH-6133
[] [] [] []
REALBUG TE29
[] [] [] []
INNOBABY TE-RING03
[] [] [] []
REALBUG TC101
[] [] [] []
TOMY T73069
[] [] [] []
TOMY T72737
[] [] [] []


In [26]:
new_logs = pd.DataFrame(output_rows)
new_logs

,brand,mpn,title,api_response,gtins,gtin_exists
0,VATERRA,VTR03090,Vaterra 1/10 1972 Chevrolet K10 Pickup Ascende...,200,[],False
1,WWE,US610203,WWE Massive Belt Banners - Airnormous WWE Unit...,200,[],False
2,SONGMICS,UGKR48WT,SONGMICS UGKR48WT Children’s Toy Storage Unit ...,200,[],False
3,WWE,TMLWWE2,WWE TeenyMates Series 2 Locker Room Set,200,[733947120073],True
4,THD,TH-6133,THD TH-6133 White Large Indoor or Outdoor Ther...,200,[],False
5,REALBUG,TE29,REALBUG Dragonfly Paperweight(3x3x1),200,[],False
6,INNOBABY,TE-RING03,INNOBABY TEETHIN' Smart Birthstone Ring TEETHE...,200,[],False
7,REALBUG,TC101,"REALBUG 1.9 "" Black Scorpion Dome Paperweight Red",200,[],False
8,TOMY,T73069,"TOMY Rapidough Game, Multi-Color",200,[],False
9,TOMY,T72737,"TOMY Puzzle Wars Games for Kids, Multicolor",200,[],False


In [27]:
# Create a new DataFrame from the list of dictionaries
new_logs = pd.DataFrame(output_rows)

new_logs.to_csv('logs.csv', index=False, mode='a', header=False)

In [28]:
# Cleanup duplicates if they exist

# Read the updated CSV file into a DataFrame
log = pd.read_csv('logs.csv')

# Drop duplicates in the "zoroNo" column, keeping the record where "gtin_exists" is True if duplicates exist
log = log.sort_values('gtin_exists', ascending=False).drop_duplicates(subset='mpn')

# Write the filtered DataFrame back to the CSV file
log.to_csv('logs.csv', index=False)

# Report on GTIN Collection

In [29]:
import plotly.express as px
log = pd.read_csv("logs.csv", index_col=False)
log

,brand,mpn,title,api_response,gtin,gtin_exists
0,WWE,TMLWWE2,WWE TeenyMates Series 2 Locker Room Set,200,['733947120073'],True
1,VATERRA,VTR03090,Vaterra 1/10 1972 Chevrolet K10 Pickup Ascende...,200,[],False
2,WWE,US610203,WWE Massive Belt Banners - Airnormous WWE Unit...,200,[],False
3,SONGMICS,UGKR48WT,SONGMICS UGKR48WT Children’s Toy Storage Unit ...,200,[],False
4,THD,TH-6133,THD TH-6133 White Large Indoor or Outdoor Ther...,200,[],False
5,REALBUG,TE29,REALBUG Dragonfly Paperweight(3x3x1),200,[],False
6,INNOBABY,TE-RING03,INNOBABY TEETHIN' Smart Birthstone Ring TEETHE...,200,[],False
7,REALBUG,TC101,"REALBUG 1.9 "" Black Scorpion Dome Paperweight Red",200,[],False
8,TOMY,T73069,"TOMY Rapidough Game, Multi-Color",200,[],False
9,TOMY,T72737,"TOMY Puzzle Wars Games for Kids, Multicolor",200,[],False


In [30]:
api_response = log.groupby(by="api_response")['mpn'].count()
api_response = api_response.rename('count').reset_index().rename(columns={'api_response': 'api_response'})

print(f"SKUs Requested: {sum(api_response['count'])}")
display(api_response)

fig = px.bar(api_response, x="api_response", y='count',title="GTIN Collection Logs | API Responses")

# Annotate each bar with the percentage
for i in range(len(api_response)):
    total_count = api_response['count'].sum()
    percentage = (api_response.iloc[i]['count'] / total_count) * 100
    fig.add_annotation(x=api_response.index[i], y=api_response.iloc[i]['count'] + 10, 
                       text=f"{percentage:.2f}%", showarrow=False)

fig.show()

SKUs Requested: 10


,api_response,count
0,200,10


In [31]:
gtin_exists = log.groupby(by="gtin_exists")['mpn'].count()
gtin_exists = gtin_exists.rename('count').reset_index().rename(columns={'gtin_exists': 'gtin_exists'})

print(f"SKUs Requested: {sum(gtin_exists['count'])}")
display(gtin_exists)

fig = px.bar(gtin_exists, x="gtin_exists", y='count',title="GTIN Collection Logs | GTINs Found")

# Annotate each bar with the percentage
for i in range(len(gtin_exists)):
    total_count = gtin_exists['count'].sum()
    percentage = (gtin_exists.iloc[i]['count'] / total_count) * 100
    fig.add_annotation(x=gtin_exists.index[i], y=gtin_exists.iloc[i]['count'] + 10, 
                       text=f"{percentage:.2f}%", showarrow=False)

fig.show()

SKUs Requested: 10


,gtin_exists,count
0,False,9
1,True,1
